# Example: Identifying Birds of Prey with CON-FOLD

This tutorial demonstrates how to use CON-FOLD, an extension of the FOLD-RM algorithm that incorporates confidence values and allows for the inclusion of expert domain knowledge.

We will use a simple (fictional) dataset of 20 birds to classify whether a bird is a predator based on two features:
- `wingspan` (a numerical feature)
- `beak` type (a categorical feature)

In [1]:
# --- Imports ---
# Make sure your project's root directory is in the Python path.
# In many setups, Jupyter does this automatically. If not, you might need:
# import sys
# sys.path.append('..') 
import sys
sys.path.append(r'...')

import numpy as np
from foldrm import Classifier
from datasets import birds # Our new function
from utils import split_data # Or your stratified version if you prefer

## Step 1: Load and Prepare the Data

First, we load our custom birds dataset using the function we added to `datasets.py`. We will then split the data into a training set (the first 15 birds) and a testing set (the last 5 birds). This fixed split ensures our results are reproducible every time.

In [2]:
# Load the data
model_template, data = birds(data_path='../data/birds/birds.csv')

# Split into training and testing sets
# We use a simple slice for reproducibility
data_train = data[:15]
data_test = data[15:]

print(f"Training set size: {len(data_train)} birds")
print(f"Testing set size: {len(data_test)} birds")


% birds dataset loaded (20, 3)
Training set size: 15 birds
Testing set size: 5 birds


## Step 2: The Baseline Model - Learning From Scratch

Let's see what the algorithm can learn on its own without any help. We will create a standard `Classifier`, train it on our 15 birds, and then see what rules it generates.

In [3]:
# Instantiate a new classifier for our baseline experiment
baseline_model = Classifier(attrs=model_template.attrs, numeric=model_template.numeric, label=model_template.label)

# Fit the model on the training data
baseline_model.fit(data_train, ratio=0.5)

# Print the rules the model learned
print("--- Rules Learned by the Baseline Model ---")
baseline_model.print_asp(simple=True)

--- Rules Learned by the Baseline Model ---
predator(X,'yes') :- not beak(X,'curved'), not ab1(X). [confidence: 0.73529]
predator(X,'no') :- wingspan(X,N0), N0>10.0. [confidence: 0.7]
predator(X,'no') :- wingspan(X,N0), N0<=10.0. [confidence: 0.55]
ab1(X) :- wingspan(X,N0), N0<=20.0.


### Evaluate the Baseline Model

Now, we'll use our 5-bird test set to see how accurate the baseline model is.

In [4]:
# Prepare the test data (features and true labels)
X_test = [d[:-1] for d in data_test]
Y_test = [d[-1] for d in data_test]

# Get predictions (these will be tuples of (label, confidence))
predictions_tuples = baseline_model.predict(X_test)
predicted_labels = [p[0] for p in predictions_tuples]

# Calculate accuracy
correct_predictions = 0
for i in range(len(Y_test)):
    if predicted_labels[i] == Y_test[i]:
        correct_predictions += 1

accuracy = correct_predictions / len(Y_test)

print("--- Baseline Model Evaluation ---")
print(f"True Labels:    {Y_test}")
print(f"Predicted Labels: {predicted_labels}")
print(f"Accuracy: {accuracy * 100:.2f}%")

--- Baseline Model Evaluation ---
True Labels:    ['yes', 'no', 'no', 'yes', 'no']
Predicted Labels: ['yes', 'no', 'yes', 'yes', 'no']
Accuracy: 80.00%


## Step 3: Injecting "Expert" Domain Knowledge

The baseline model was good, but not perfect. It struggled to correctly identify small birds with sharp beaks. This is a perfect opportunity to improve performance by providing some "expert" knowledge from an ornithologist. This is the core feature of CON-FOLD.

Instead of defining all the rules manually, we will only provide the most nuanced and difficult piece of information—the rule that the baseline model got wrong.

Our expert tells us:
- **"A bird with a sharp beak is a predator, UNLESS it is very small (with a wingspan of 25cm or less)."**

By providing only this complex exception rule, we give the model a powerful head start. We can then let CON-FOLD learn the more straightforward rules (like what to do about birds with curved beaks) from the data on its own. Let's translate this one key insight into CON-FOLD's rule syntax.

In [5]:
# Instantiate a new classifier for our expert-guided model
expert_model = Classifier(attrs=model_template.attrs, numeric=model_template.numeric, label=model_template.label)

# Define our expert rules as strings
# Note: the symbols '==' and '<=' must also be in single quotes for the parser.
rule1 = "with confidence 0.95 class = 'yes' if 'beak' '==' 'Sharp' except if 'wingspan' '<=' '25'"
#Note additional rules could be added like this:
#rule2 = "with confidence 0.99 class = 'no' if 'beak' '==' 'Curved'"

# Add the manual rules to the model
expert_model.add_manual_rule(rule1, model_template.attrs, model_template.numeric, ['yes', 'no'], instructions=False)
# Note: here is code to add an additional rule:
#expert_model.add_manual_rule(rule2, model_template.attrs, model_template.numeric, ['yes', 'no'], instructions=False)

print("--- Manual Rules Added to the Model (Before Training) ---")
# The internal representation is a bit complex, but we can see our rules are in there.
for rule in expert_model.rules:
    print(rule)

--- Manual Rules Added to the Model (Before Training) ---
((-1, '==', 'yes'), [(1, '==', 'Sharp')], [(-1, [(0, '<=', 25)], [], 0)], 0.95)


## Step 4: Train the Expert-Guided Model

Now that our expert knowledge is locked in, we will train the model on the exact same training data. 

CON-FOLD is smart enough to use these rules as a starting point. It will now only try to learn new rules for any training examples that our manual rules didn't already cover. This makes its job much easier and less prone to errors.

In [6]:
# Now, fit the model on the training data.
# The algorithm will work around the rules we provided.
expert_model.fit(data_train, ratio=0.75)

# Print the final, combined rule set
print("--- Final Ruleset from the Expert Model ---")
expert_model.print_asp(simple=True)

--- Final Ruleset from the Expert Model ---
predator(X,'yes') :- beak(X,'sharp'), not ab1(X). [confidence: 0.95]
predator(X,'no') :- wingspan(X,N0), N0>10.0. [confidence: 0.7]
predator(X,'no') :- wingspan(X,N0), N0<=10.0. [confidence: 0.55]
ab1(X) :- wingspan(X,N0), N0<=25.


## Step 5: Compare the Results

Let's evaluate our new expert-guided model on the same 5-bird test set. Did our domain knowledge help improve the accuracy?

In [7]:
# Get predictions from our new model
expert_predictions_tuples = expert_model.predict(X_test)
expert_predicted_labels = [p[0] for p in expert_predictions_tuples]

# Calculate accuracy
expert_correct_predictions = 0
for i in range(len(Y_test)):
    if expert_predicted_labels[i] == Y_test[i]:
        expert_correct_predictions += 1

expert_accuracy = expert_correct_predictions / len(Y_test)

print("--- Baseline Model Evaluation ---")
print(f"True Labels:      {Y_test}")
print(f"Predicted Labels: {predicted_labels}")
print(f"Accuracy: {accuracy * 100:.2f}%\n")


print("--- Expert Model Evaluation ---")
print(f"True Labels:      {Y_test}")
print(f"Predicted Labels: {expert_predicted_labels}")
print(f"Accuracy: {expert_accuracy * 100:.2f}%")

--- Baseline Model Evaluation ---
True Labels:      ['yes', 'no', 'no', 'yes', 'no']
Predicted Labels: ['yes', 'no', 'yes', 'yes', 'no']
Accuracy: 80.00%

--- Expert Model Evaluation ---
True Labels:      ['yes', 'no', 'no', 'yes', 'no']
Predicted Labels: ['yes', 'no', 'no', 'yes', 'no']
Accuracy: 100.00%


## Step 6: Adding Expert Rules and Learning Their Confidence

A common scenario is that an expert knows a rule is generally true, but doesn't know the exact statistics or confidence.

CON-FOLD is designed for this. We can provide rules without a confidence value, and the algorithm will **calculate the confidence for us** based on the training data.

Let's provide two rules our ornithologist is fairly certain about:
1.  **"Birds with a curved beak are not predators."**
2.  **"A bird with a sharp beak is a predator, UNLESS it is very small (wingspan <= 25cm)."**

In [8]:
# Instantiate a new classifier
learned_confidence_model = Classifier(attrs=model_template.attrs, numeric=model_template.numeric, label=model_template.label)

# Define our expert rules as strings, but WITHOUT the 'with confidence' part.
rule1_no_confidence = "class = 'no' if 'beak' '==' 'Curved'"
rule2_no_confidence = "class = 'yes' if 'beak' '==' 'Sharp' except if 'wingspan' '<=' '25'"

# Add the manual rules to the model
learned_confidence_model.add_manual_rule(rule1_no_confidence, model_template.attrs, model_template.numeric, ['yes', 'no'], instructions=False)
learned_confidence_model.add_manual_rule(rule2_no_confidence, model_template.attrs, model_template.numeric, ['yes', 'no'], instructions=False)

print("--- Manual Rules Added (Before Training) ---")
print("Notice the default confidence value of 0.5 assigned to each rule.")
for rule in learned_confidence_model.rules:
    print(rule)

--- Manual Rules Added (Before Training) ---
Notice the default confidence value of 0.5 assigned to each rule.
((-1, '==', 'no'), [(1, '==', 'Curved')], [], 0.5)
((-1, '==', 'yes'), [(1, '==', 'Sharp')], [(-1, [(0, '<=', 25)], [], 0)], 0.5)


### Learning the Confidence from Data

As we saw above, the rules were added with a placeholder confidence of `0.5`. Now, when we call `.fit()`, CON-FOLD will evaluate these rules against the training data and replace the placeholder with a properly calculated confidence score.

In [9]:
# Now, fit the model on the training data.
# The algorithm will calculate the confidence of our provided rules and then learn any additional rules needed.
learned_confidence_model.fit(data_train, ratio=0.5)

# Print the final, combined rule set
print("--- Final Ruleset with Learned Confidence ---")
print("The confidence values have now been updated based on the training data!")
learned_confidence_model.print_asp(simple=True)
#Note that confidence values will be relatively low due to the small size of the training data. 

--- Final Ruleset with Learned Confidence ---
The confidence values have now been updated based on the training data!
predator(X,'no') :- beak(X,'curved'). [confidence: 0.65385]
predator(X,'yes') :- beak(X,'sharp'), not ab1(X). [confidence: 0.73529]
predator(X,'no') :- wingspan(X,N0), N0>10.0. [confidence: 0.59091]
predator(X,'no') :- wingspan(X,N0), N0<=10.0. [confidence: 0.55]
ab1(X) :- wingspan(X,N0), N0<=25.


In [10]:
# Get predictions from our new model
learned_conf_predictions = learned_confidence_model.predict(X_test)
learned_conf_labels = [p[0] for p in learned_conf_predictions]

# Calculate accuracy
learned_conf_accuracy = sum(1 for i in range(len(Y_test)) if learned_conf_labels[i] == Y_test[i]) / len(Y_test)

print("--- Learned Confidence Model Evaluation ---")
print(f"True Labels:      {Y_test}")
print(f"Predicted Labels: {learned_conf_labels}")
print(f"Accuracy: {learned_conf_accuracy * 100:.2f}%")

--- Learned Confidence Model Evaluation ---
True Labels:      ['yes', 'no', 'no', 'yes', 'no']
Predicted Labels: ['yes', 'no', 'no', 'yes', 'no']
Accuracy: 100.00%


## Step 7: Pruning for a Simpler, More Robust Model

Our expert-guided model is 100% accurate on the test set, which is fantastic. However, in larger, noisier datasets, the algorithm might still learn some overly specific rules that have low confidence.

CON-FOLD provides two methods to handle this:
1.  **Simple Confidence Pruning:** After training, we can remove any rule that doesn't meet a minimum confidence threshold.
2.  **Confidence-Driven Learning:** We can use a more advanced training method (`confidence_fit`) that only adds exceptions to rules if they provide a significant boost in confidence.

Let's explore both.

### Method 1: Simple Post-Hoc Confidence Pruning

This is the most straightforward approach. We'll take our trained `expert_model` and apply a pruning function to remove any rules with a confidence below a certain threshold, say `0.90`.

In [11]:
# First, let's re-print the rules from our expert model for comparison
print("--- Rules Before Pruning ---")
expert_model.print_asp(simple=True)

# Import the prune_rules function from the core algorithm file
from algo import prune_rules

# Apply the pruning function
# This will create a new list containing only the rules that meet the confidence threshold.
pruned_rules = prune_rules(expert_model.rules, confidence=0.70)

# We can create a new model instance to hold these pruned rules
simple_pruned_model = Classifier(attrs=model_template.attrs, numeric=model_template.numeric, label=model_template.label)
simple_pruned_model.rules = pruned_rules

print("\n--- Rules After Pruning (Confidence >= 0.70) ---")
simple_pruned_model.print_asp(simple=True)

--- Rules Before Pruning ---
predator(X,'yes') :- beak(X,'sharp'), not ab1(X). [confidence: 0.95]
predator(X,'no') :- wingspan(X,N0), N0>10.0. [confidence: 0.7]
predator(X,'no') :- wingspan(X,N0), N0<=10.0. [confidence: 0.55]
ab1(X) :- wingspan(X,N0), N0<=25.

--- Rules After Pruning (Confidence >= 0.70) ---
predator(X,'yes') :- beak(X,'sharp'), not ab1(X). [confidence: 0.95]
predator(X,'no') :- wingspan(X,N0), N0>10.0. [confidence: 0.7]
ab1(X) :- wingspan(X,N0), N0<=25.


### Method 2: Advanced Confidence-Driven Learning

A more powerful method is to integrate the idea of confidence directly into the learning process. The `confidence_fit` method does exactly this.

It will **both calculate the confidence** of any manually provided rules (if not specified) and **prevent the model from adding insignificant exceptions** during its own learning process.

Let's train a new model using this method on our two expert rules (for which we did not provide confidence). We will set a very high `improvement_threshold` of `0.15` (or 15%). This means the algorithm will be very strict and will only add a new exception to a rule if it improves the confidence by at least 15%, effectively pruning weak exceptions before they are even added.

In [12]:
# Instantiate a new model for this experiment
advanced_pruning_model = Classifier(attrs=model_template.attrs, numeric=model_template.numeric, label=model_template.label)

# Add our same expert rules WITHOUT confidence
#rule1_no_confidence = "class = 'no' if 'beak' '==' 'Curved'"
#rule2_no_confidence = "class = 'yes' if 'beak' '==' 'Sharp' except if 'wingspan' '<=' '25'"
#advanced_pruning_model.add_manual_rule(rule1_no_confidence, model_template.attrs, model_template.numeric, ['yes', 'no'], instructions=False)
#advanced_pruning_model.add_manual_rule(rule2_no_confidence, model_template.attrs, model_template.numeric, ['yes', 'no'], instructions=False)

# Now, train using confidence_fit with a high 15% improvement threshold
print("--- Training with confidence_fit(improvement_threshold=0.15) ---")
advanced_pruning_model.confidence_fit(data_train, improvement_threshold=0.15)

print("\n--- Rules Learned via Confidence-Driven Learning ---")
print("Note how the model is simpler and did not learn any exceptions to rules or `abnormalities', as they did not meet the high confidence improvement threshold.")
advanced_pruning_model.print_asp(simple=True)

--- Training with confidence_fit(improvement_threshold=0.15) ---

--- Rules Learned via Confidence-Driven Learning ---
Note how the model is simpler and did not learn any exceptions to rules or `abnormalities', as they did not meet the high confidence improvement threshold.
predator(X,'yes') :- not beak(X,'curved'). [confidence: 0.625]
predator(X,'no') :- wingspan(X,N0), N0>30.0. [confidence: 0.625]
predator(X,'no') :- wingspan(X,N0), N0<=30.0. [confidence: 0.55]
